In [24]:
# from minio.error import S3Error
from minio import Minio

minio_url = 'minio:9000'
access_key = "Qm6pkTNzxbz3Km6QHVwc"
secret_key = "wa3LjHTKv3kQOEg8SQjSivwKinh6PKRVZlnYkUbY"

minio_client = Minio(endpoint=minio_url,
                access_key=access_key,
                secret_key=secret_key,
                secure=False)

b_name = 'warehouse'

objects = minio_client.list_objects(
                b_name, recursive=True, prefix="images/albums/",
            )

o_names = []
for obj in objects:
    o_names.append(obj.object_name)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Format_data").getOrCreate()

23/11/06 06:48:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
%%sql
SELECT * FROM gold.data_for_recommendation ORDER BY popularity DESC

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


track_id,track_name,artist_id,album_id,album_image,artist_image,track_audio,release_date,artist_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
6VXVYATpQXEIoZ97NnWCmn,Vì Anh Đâu Có Biết,1OIYKbmhG0RxPXvVPNj3NN,3C5z2dMU7D6VLpnNtYujPJ,images/albums/2022/8/3C5z2dMU7D6VLpnNtYujPJ.jpg,images/artists/2022/8/1OIYKbmhG0RxPXvVPNj3NN.jpg,audios/2022/8/6VXVYATpQXEIoZ97NnWCmn.mp3,2022-08-04,Madihu,69.0,0.8610000014305115,0.7749999761581421,241920,0.3580000102519989,0.0,9,0.41499999165534973,-6.4730000495910645,bytearray(b'0'),0.05270000174641609,139.98199462890625,0.5239999890327454
6VXVYATpQXEIoZ97NnWCmn,Vì Anh Đâu Có Biết,57g2v7gJZepcwsuwssIfZs,3C5z2dMU7D6VLpnNtYujPJ,images/albums/2022/8/3C5z2dMU7D6VLpnNtYujPJ.jpg,images/artists/2022/8/57g2v7gJZepcwsuwssIfZs.jpg,audios/2022/8/6VXVYATpQXEIoZ97NnWCmn.mp3,2022-08-04,Vũ.,69.0,0.8610000014305115,0.7749999761581421,241920,0.3580000102519989,0.0,9,0.41499999165534973,-6.4730000495910645,bytearray(b'0'),0.05270000174641609,139.98199462890625,0.5239999890327454
255vSRpVq5YYKBJiem1BVx,Chơi Như Tụi Mỹ,4grjJqg7iwQ8RKHs8d9Snh,7qPMuPHg2shbQIGyxOd09A,images/albums/2022/12/7qPMuPHg2shbQIGyxOd09A.jpg,images/artists/2022/12/4grjJqg7iwQ8RKHs8d9Snh.jpg,audios/2022/12/255vSRpVq5YYKBJiem1BVx.mp3,2022-12-30,Andree Right Hand,67.0,0.5370000004768372,0.8159999847412109,160848,0.5350000262260437,0.0,5,0.1550000011920929,-7.35099983215332,bytearray(b'0'),0.0714000016450882,108.96299743652344,0.48399999737739563
5iZHnufFUgATzrpGgH1K0W,Ánh Sao Và Bầu Trời,2rVYwIYNEsMxtt8j5yemHb,1JLUfM3IFpFyBSyhNQWkYw,images/albums/2021/9/1JLUfM3IFpFyBSyhNQWkYw.jpg,images/artists/2021/9/2rVYwIYNEsMxtt8j5yemHb.jpg,audios/2021/9/5iZHnufFUgATzrpGgH1K0W.mp3,2021-09-10,T.R.I,66.0,0.8759999871253967,0.6620000004768372,260571,0.20499999821186066,0.003539999946951866,5,0.10899999737739563,-14.005000114440918,bytearray(b'0'),0.0348999984562397,139.91099548339844,0.2680000066757202
2QqUfCa7GbxQm6gB3K4hhi,See Tình - Cukak Remix,0r63ReVRjxrS4ATbLrdcrL,3IvN0hSsaCpMIigGZ4yJ0z,images/albums/2022/5/3IvN0hSsaCpMIigGZ4yJ0z.jpg,images/artists/2022/5/0r63ReVRjxrS4ATbLrdcrL.jpg,audios/2022/5/2QqUfCa7GbxQm6gB3K4hhi.mp3,2022-05-05,Hoàng Thùy Linh,66.0,0.20800000429153442,0.8740000128746033,170159,0.7459999918937683,0.006899999920278788,0,0.07959999889135361,-6.197000026702881,bytearray(b'0'),0.04490000009536743,128.00900268554688,0.8840000033378601
2QqUfCa7GbxQm6gB3K4hhi,See Tình - Cukak Remix,3swW6OR2g7qTY3626sqVW4,3IvN0hSsaCpMIigGZ4yJ0z,images/albums/2022/5/3IvN0hSsaCpMIigGZ4yJ0z.jpg,images/artists/2022/5/3swW6OR2g7qTY3626sqVW4.jpg,audios/2022/5/2QqUfCa7GbxQm6gB3K4hhi.mp3,2022-05-05,Cukak,66.0,0.20800000429153442,0.8740000128746033,170159,0.7459999918937683,0.006899999920278788,0,0.07959999889135361,-6.197000026702881,bytearray(b'0'),0.04490000009536743,128.00900268554688,0.8840000033378601
27Z9T0mSWHYWSj9SNOGvyt,Thằng Điên,3rjcQ5VIWCN4q7UFetzdeO,0t2YXNqz7bv7n6mPorj8HC,images/albums/2022/7/0t2YXNqz7bv7n6mPorj8HC.jpg,images/artists/2022/7/3rjcQ5VIWCN4q7UFetzdeO.jpg,audios/2022/7/27Z9T0mSWHYWSj9SNOGvyt.mp3,2022-07-11,JustaTee,65.0,0.3619999885559082,0.6259999871253967,234000,0.503000020980835,1.4199999895936344e-06,5,0.07620000094175339,-8.753000259399414,bytearray(b'1'),0.03889999911189079,140.01400756835938,0.5740000009536743
27Z9T0mSWHYWSj9SNOGvyt,Thằng Điên,5Kh0ta0UY4uJ4g2CIdq9V9,0t2YXNqz7bv7n6mPorj8HC,images/albums/2022/7/0t2YXNqz7bv7n6mPorj8HC.jpg,images/artists/2022/7/5Kh0ta0UY4uJ4g2CIdq9V9.jpg,audios/2022/7/27Z9T0mSWHYWSj9SNOGvyt.mp3,2022-07-11,Phương Ly,65.0,0.3619999885559082,0.6259999871253967,234000,0.503000020980835,1.4199999895936344e-06,5,0.07620000094175339,-8.753000259399414,bytearray(b'1'),0.03889999911189079,140.01400756835938,0.5740000009536743
6IUEwlNgVnT5CYeBuq6DTH,"Anh Là Ai? (feat. Dick, DT & UMIE)",1cPpd989kghzlZqfpP4KwL,6t6MpiPsDCgZutjx2BQF5Z,images/albums/2023/7/6t6MpiPsDCgZutjx2BQF5Z.jpg,images/artists/2023/7/1cPpd989kghzlZqfpP4KwL.jpg,audios/2023

In [8]:
%%sql
ALTER TABLE gold.data_for_recommendation
ADD COLUMN album_id string AFTER artist_id

23/11/06 06:52:50 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [14]:
%%sql
MERGE INTO gold.data_for_recommendation as target
USING (SELECT DISTINCT track_id, album_id FROM silver.spotify_tracks) as source
    ON target.track_id = source.track_id
WHEN MATCHED
    THEN UPDATE SET target.album_id = source.album_id

++
||
++
++

In [18]:
%%sql
ALTER TABLE gold.data_for_recommendation
ADD COLUMN album_image string AFTER album_id

23/11/06 07:22:09 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [27]:
%%sql
ALTER TABLE gold.data_for_recommendation
ADD COLUMN release_date date AFTER album_image

23/11/06 07:48:18 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [ ]:
%%sql
MERGE INTO gold.data_for_recommendation as target
USING (SELECT DISTINCT track_id, release_date FROM silver.spotify_tracks) as source
    ON target.track_id = source.track_id
WHEN MATCHED
    THEN UPDATE SET target.release_date = source.release_date

In [153]:
# from pyspark.sql.functions import concat_ws,col,lit
release_df = spark.sql("SELECT DISTINCT album_id, year(release_date) as release_year , month(release_date) as release_month FROM gold.data_for_recommendation")
# release_df = release_df.withColumn('prefix_image', lit('images/albums'))
# release_df  = release_df.withColumn("album_image", concat_ws('/',release_df.prefix_image, release_df.release_year, release_df.release_month, release_df.album_id))
release_df.createOrReplaceTempView("release_view")
release_df.show()

+--------------------+------------+-------------+
|            album_id|release_year|release_month|
+--------------------+------------+-------------+
|6GHBzU9XBSBI4miS5...|        1900|            1|
|5AOBVcOjv2UkGS0bf...|        1990|            9|
|6ObGIW3iwi4krD30P...|        1997|            6|
|2tER8kzLUxSScpGDK...|        1997|            6|
|2zjxRD09aVT2hX2vO...|        1997|            6|
|6oPlqW7j9KngohGq1...|        1998|            9|
|0uRy8ZtJBJ1hGom1W...|        1998|            1|
|6bVTbwSRweBfugLld...|        1999|            9|
|1yzcg4Z4GXRGBOclt...|        1999|            1|
|2OlX9no9oYv2mivzs...|        1999|            9|
|0c8ol9E8xImZCieHu...|        1999|            1|
|0Aq8Njk7QvZAJWl12...|        1999|            9|
|5oj0nMY6mwm6xyhwX...|        2000|           10|
|0sTHiLQW4Z4NUISpR...|        2000|            9|
|6YCHRe265jtOesblV...|        2000|            9|
|3Ym0jiFd1KZ2JSVnL...|        2001|            1|
|7Dnja3CqNJBigoLGR...|        2001|            6|


In [155]:
%%sql
MERGE INTO gold.data_for_recommendation as target
USING (SELECT * FROM release_view) as source
    ON target.album_id = source.album_id
WHEN MATCHED
    THEN UPDATE SET target.album_image = CONCAT('images/albums/', source.release_year, '/', source.release_month, '/', source.album_id,'.jpg')

++
||
++
++

Insert Artist Image

In [163]:
%%sql
ALTER TABLE gold.data_for_recommendation
ADD COLUMN artist_image string AFTER album_image

23/11/06 09:41:18 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [173]:
%%sql
MERGE INTO gold.data_for_recommendation as target
USING (SELECT DISTINCT track_id, artist_id, year(release_date) as release_year , month(release_date) as release_month FROM gold.data_for_recommendation) as source
    ON target.track_id = source.track_id and target.artist_id = source.artist_id
WHEN MATCHED
    THEN UPDATE SET target.artist_image = CONCAT('images/artists/', source.release_year, '/', source.release_month, '/', source.artist_id,'.jpg')

++
||
++
++

In [174]:
%%sql
ALTER TABLE gold.data_for_recommendation
ADD COLUMN track_audio string AFTER artist_image

23/11/06 10:12:35 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


++
||
++
++

In [175]:
%%sql
MERGE INTO gold.data_for_recommendation as target
USING (SELECT DISTINCT track_id, artist_id, year(release_date) as release_year , month(release_date) as release_month FROM gold.data_for_recommendation) as source
    ON target.track_id = source.track_id and target.artist_id = source.artist_id
WHEN MATCHED
    THEN UPDATE SET target.track_audio = CONCAT('audios/', source.release_year, '/', source.release_month, '/', source.track_id,'.mp3')

++
||
++
++